<a href="https://colab.research.google.com/github/Yassmina-Abdo/realtime-head-pose-estimation/blob/main/Notebooks/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

^C
Note: you may need to restart the kernel to use updated packages.


## Liberaries

In [1]:
import numpy as np
import pickle
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR, SVR,SVC
from sklearn.preprocessing import MinMaxScaler,StandardScaler
# from xgboost.sklearn import XGBRegressor
###########
import cv2
import mediapipe as mp
import numpy as np
import time
import pandas
import os
import cv2
import mediapipe as mp
import numpy as np
import time
import pickle
from math import cos, sin
from sklearn.preprocessing import MinMaxScaler,StandardScaler


## Reading the prepared dataset and split it

In [2]:
path_local='../Dataset/Data/data468_normed_low.csv'
path_drive='/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Data/media_with_angles_low.csv'
data =pd.read_csv(path_local)
data.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y463,y464,y465,y466,y467,y468,yaw,pitch,roll,image_name
0,0.0,0.768221,0.042679,0.298753,0.896258,0.768221,0.256074,-2.987527,0.256074,0.384111,...,-22.0,-86.0,-83.0,-80.0,-91.0,-95.0,0.725284,-0.140406,-0.174674,image03195
1,0.0,-0.119313,-0.053028,-0.198854,-0.132570,-0.132570,-0.092799,-0.755647,-0.106056,-0.119313,...,-15.0,-74.0,-72.0,-70.0,-84.0,-86.0,-0.233745,-0.116700,0.095809,image03247
2,0.0,-0.257694,0.000000,-0.227377,-0.303170,-0.257694,-0.090951,-0.212219,-0.090951,-0.151585,...,-23.0,-76.0,-72.0,-70.0,-86.0,-92.0,-1.330352,0.003072,0.362599,image03214
3,0.0,-0.011573,0.023147,-0.081013,0.000000,0.000000,0.046293,-0.763836,0.057866,0.057866,...,-21.0,-105.0,-99.0,-94.0,-121.0,-125.0,-0.040907,-0.474087,0.019801,image03238
4,0.0,0.126856,0.063428,0.047571,0.158570,0.174427,0.206142,-0.856280,0.237856,0.269570,...,-24.0,-94.0,-91.0,-88.0,-99.0,-102.0,0.205732,-0.241359,-0.150232,image03236


In [3]:
X = data.iloc[:,:-4]
X

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y459,y460,y461,y462,y463,y464,y465,y466,y467,y468
0,0.0,0.768221,0.042679,0.298753,0.896258,0.768221,0.256074,-2.987527,0.256074,0.384111,...,-24.0,-27.0,-24.0,-23.0,-22.0,-86.0,-83.0,-80.0,-91.0,-95.0
1,0.0,-0.119313,-0.053028,-0.198854,-0.132570,-0.132570,-0.092799,-0.755647,-0.106056,-0.119313,...,-17.0,-20.0,-22.0,-16.0,-15.0,-74.0,-72.0,-70.0,-84.0,-86.0
2,0.0,-0.257694,0.000000,-0.227377,-0.303170,-0.257694,-0.090951,-0.212219,-0.090951,-0.151585,...,-25.0,-28.0,-23.0,-25.0,-23.0,-76.0,-72.0,-70.0,-86.0,-92.0
3,0.0,-0.011573,0.023147,-0.081013,0.000000,0.000000,0.046293,-0.763836,0.057866,0.057866,...,-22.0,-26.0,-30.0,-21.0,-21.0,-105.0,-99.0,-94.0,-121.0,-125.0
4,0.0,0.126856,0.063428,0.047571,0.158570,0.174427,0.206142,-0.856280,0.237856,0.269570,...,-26.0,-29.0,-25.0,-25.0,-24.0,-94.0,-91.0,-88.0,-99.0,-102.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1730,0.0,0.285456,0.109791,0.241540,0.351330,0.373288,0.395247,-0.922242,0.483079,0.548954,...,-19.0,-21.0,-18.0,-18.0,-17.0,-76.0,-74.0,-72.0,-76.0,-77.0
1731,0.0,-0.335925,-0.063986,-0.335925,-0.367918,-0.351921,-0.191957,-0.399910,-0.191957,-0.239946,...,-21.0,-23.0,-19.0,-20.0,-18.0,-73.0,-71.0,-69.0,-80.0,-85.0
1732,0.0,0.015067,0.015067,-0.030134,0.030134,0.060268,0.105469,-0.738283,0.150670,0.165737,...,-14.0,-16.0,-17.0,-13.0,-12.0,-69.0,-68.0,-66.0,-73.0,-75.0
1733,0.0,0.499026,0.124757,0.249513,0.530215,0.499026,0.343081,-1.559457,0.374270,0.436648,...,-18.0,-20.0,-14.0,-17.0,-16.0,-64.0,-62.0,-60.0,-61.0,-64.0


In [4]:
y = data.iloc[:,-4:-1]
y.head(1)

,yaw,pitch,roll
0,0.725284,-0.140406,-0.174674


## Scaling

In [5]:
# scaler = StandardScaler()
# scaler = scaler.fit(X.to_numpy())
# x_scaled= scaler.transform(X.to_numpy())

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X.to_numpy(), y.to_numpy(), test_size=0.20, random_state=42, shuffle=True)

In [7]:
x_train

array([[ 0.00000000e+00, -1.24327846e-01, -1.06566725e-01, ...,
        -6.80000000e+01, -9.10000000e+01, -9.60000000e+01],
       [ 0.00000000e+00, -5.80693504e-02, -9.67822507e-02, ...,
        -6.50000000e+01, -7.90000000e+01, -8.30000000e+01],
       [ 0.00000000e+00,  1.54211585e-02,  1.54211585e-02, ...,
        -9.30000000e+01, -1.04000000e+02, -1.06000000e+02],
       ...,
       [ 0.00000000e+00,  1.05409255e-01,  3.01169301e-02, ...,
        -8.10000000e+01, -9.80000000e+01, -1.01000000e+02],
       [ 0.00000000e+00, -2.68293281e-02,  5.36586563e-02, ...,
        -6.30000000e+01, -6.40000000e+01, -7.00000000e+01],
       [ 0.00000000e+00,  1.87949200e-01,  6.26497334e-02, ...,
        -6.60000000e+01, -8.20000000e+01, -8.50000000e+01]])

## **Trial #1:**

- MultiOutputRegressor with RandomForestRegressor


> https://www.r-bloggers.com/2021/10/multi-output-regression-using-sklearn/





In [ ]:
clf = MultiOutputRegressor(RandomForestRegressor(max_depth=12, random_state=42,n_estimators=300,min_samples_split=4))
clf.fit(x_train, y_train)
clf.score(x_train, y_train, sample_weight=None)

In [ ]:
y_pred= clf.predict(x_test)
clf.score(x_test, y_test, sample_weight=None)

In [ ]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_Rm.pkl', 'wb') as file:
        pickle.dump(clf, file)

## **Trial #2:**
  - SVM

  - https://machinelearningmastery.com/multi-output-regression-models-with-python/

In [ ]:
# define base model
model = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,coef0=1)
# define the direct multioutput wrapper model
wrapper = MultiOutputRegressor(model)
wrapper.fit(x_train, y_train)
wrapper.predict(x_test)

In [ ]:
print(wrapper.score(x_train, y_train, sample_weight=None))
print(wrapper.score(x_test, y_test, sample_weight=None))

In [ ]:
# save model
with open('../SavedModels/model_svm.pkl', 'wb') as file:
        pickle.dump(wrapper, file)

## **Trial #3:**

- GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=42,n_estimators=100,learning_rate=0.05)
model = MultiOutputRegressor(estimator=gbr)

In [ ]:
model.fit(x_train, y_train)
score = model.score(x_train, y_train)
print("Training score:", score)
ypred = model.predict(x_test)
model.score(x_test, y_test, sample_weight=None)

In [ ]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_GB.pkl', 'wb') as file:
        pickle.dump(model, file)

In [10]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width /2
        tdy = height /2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img


In [11]:
def yaw2rotmat(yaw,roll,pitch):
    x = pitch
    y = roll
    z = -yaw
    ch = np.cos(z)
    sh = np.sin(z)
    ca = np.cos(y)
    sa = np.sin(y)
    cb = np.cos(x)
    sb = np.sin(x)
    rot = np.zeros((3,3), 'float32')
    rot[0][0] = ch * ca
    rot[0][1] = sh*sb - ch*sa*cb
    rot[0][2] = ch*sa*sb + sh*cb
    rot[1][0] = sa
    rot[1][1] = ca * cb
    rot[1][2] = -ca * sb
    rot[2][0] = -sh * ca
    rot[2][1] = sh*sa*cb + ch*sb
    rot[2][2] = -sh*sa*sb + ch*cb
    return rot

In [12]:
def roll2rotmat(yaw):
    x = 0.0
    y = yaw
    z = 0.0
    ch = np.cos(z)
    sh = np.sin(z)
    ca = np.cos(y)
    sa = np.sin(y)
    cb = np.cos(x)
    sb = np.sin(x)
    rot = np.zeros((3,3), 'float32')
    rot[0][0] = ch * ca
    rot[0][1] = sh*sb - ch*sa*cb
    rot[0][2] = ch*sa*sb + sh*cb
    rot[1][0] = sa
    rot[1][1] = ca * cb
    rot[1][2] = -ca * sb
    rot[2][0] = -sh * ca
    rot[2][1] = sh*sa*cb + ch*sb
    rot[2][2] = -sh*sa*sb + ch*cb
    return rot

In [15]:
pkl_filename='model_RM.pkl'
with open(pkl_filename, 'rb') as file:
    pretrained_model = pickle.load(file)

## Try everything 

In [ ]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)


mp_drawing = mp.solutions.drawing_utils

drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

cap = cv2.VideoCapture(0)
i = 0

while cap.isOpened():
    success, image = cap.read()

    start = time.time()

    # Flip the image horizontally for a later selfie-view display
    # Also convert the color space from BGR to RGB
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

    # To improve performance
    image.flags.writeable = False
    
    # Get the result
    results = face_mesh.process(image)
    
    # To improve performance
    image.flags.writeable = True
    
    # Convert the color space from RGB to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    img_h, img_w, img_c = image.shape
    face_3d = []
    face_2d = []
    x_points=[]
    y_points=[]
 

    if results.multi_face_landmarks:   
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                # if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                    if idx == 1 :
                        nose_2d = (lm.x * img_w, lm.y * img_h)
                        nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)
                    elif idx == 428 :
                        chin_2d = (lm.x * img_w, lm.y * img_h)
                        
                        
                        


                    x, y = int(lm.x * img_w), int(lm.y * img_h)
                    
#                     distX = nose_2d[0] #- chin_2d[0]
#                     distY = nose_2d[1] #- chin_2d[1]
#                     dist = (distX**2 + distY**2)**0.5


                    # Get the 2D Coordinates
#                     x_points.append((x - nose_2d[0]) / dist ) 
#                     y_points.append((y- nose_2d[1]) /  dist ) 
                    x_points.append(x) 
                    y_points.append(y) 
                    
                    face_2d.append([x, y])

                    # Get the 3D Coordinates
                    face_3d.append([x, y, lm.z])       
        # Convert it to the NumPy array
        distX =   chin_2d[0]-nose_2d[0] 
        distY = chin_2d[1]  -nose_2d[1]
        dist = (distX**2 + distY**2)**0.5

        face_2d = np.array(face_2d, dtype=np.float64)
        x_point=np.array(np.transpose(x_points).astype('float'),dtype=np.float64).reshape(1,-1)
        y_point=np.array(np.transpose(y_points).astype('float'),dtype=np.float64).reshape(1,-1)
        x_point = (x_point-nose_2d[0])/dist
        y_point = (y_point-nose_2d[0])/dist
        new_data = np.concatenate((x_point,y_point),axis=1)
        
#         x_scaled = caler.transform(new_data)
        i+=1
        print(f"**********************************\n frame {i}\n {face_2d[0]}\n**************************************")
        # Convert it to the NumPy array
        face_3d = np.array(face_3d, dtype=np.float64)

        # The camera matrix
        focal_length = 1 * img_w

        cam_matrix = np.array([ [focal_length, 0, img_h / 2],
                                [0, focal_length, img_w / 2],
                                [0, 0, 1]])
        # # The distortion parameters
        dist_matrix = np.zeros((4, 1), dtype=np.float64)

#         # # Solve PnP
#         success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)
        
#         # # Get rotational matrix
#         rmat, jac = cv2.Rodrigues(rot_vec)

        # Load pretrained model
        
        angles = pretrained_model.predict(new_data)
        print(angles)
        x = angles[0][0] 
        y = angles[0][1] 
        z = angles[0][2] 
        axis = np.float32([[0.0, 0.0, 0.0], 
                             [0.0, 0.0, 0.0], 
                             [0.0, 0.0, 0.5]])

        tvec = np.array([0.0, 0.0, 1.0], np.float) # translation vector

        rot_matrix = yaw2rotmat(x,y,z)
        rvec, jacobian = cv2.Rodrigues(rot_matrix)
        imgpts, jac = cv2.projectPoints(axis, rvec, tvec, cam_matrix, dist_matrix)

        # image = draw_axis(image,x,y,z)
        # # Get angles
        # angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)
        # axis = np.float32([[0.0, 0.0, 0.0], 
        #                  [0.0, 0.0, 0.0], 
        #                  [0.0, 0.0, 0.5]])
        # x = angles[0] *360 
        # y = angles[1] *360
        # z = angles[2] *360
        
        # tvec = np.array([0.0, 0.0, 1.0], np.float64) # translation vector

        # rot_matrix = yaw2rotmat(-x,-y,-z)
        # rvec, jacobian = cv2.Rodrigues(rot_matrix)
        # imgpts, jac = cv2.projectPoints(axis, rvec, tvec, cam_matrix, dist_matrix)
        # # # Get the y rotation degree
        # x = angles[0] * 360
        # y = angles[1] * 360
        # z = angles[2] * 360
        
        
        # See where the user's head tilting
        if y < -10:
            text = "Looking Left"
        elif y > 10:
            text = "Looking Right"
        elif x < -10:
            text = "Looking Down"
        elif x > 10:
            text = "Looking Up"
        else:
            text = "Forward"

        # Display the nose direction
        # nose_3d_projection, jacobian = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)

#         p1 = (int(nose_2d[0]), int(nose_2d[1]))
#         p2 = ( int(imgpts[2][0][0]+nose_2d[0] ), int( imgpts[2][0][1] +nose_2d[1] ) )
#         p3 = ( int(imgpts[1][0][0]+nose_2d[0] ), int( imgpts[1][0][1] +nose_2d[1] ) )

#         p2 = (int(nose_2d[0] + y * 10) , int(nose_2d[1] - x * 10))
#         p3 = (int(nose_2d[0] + z * 10) , int(nose_2d[1] - x * 10))
        # # p2 = (int(imgpts[2][0][0]), int(imgpts[2][0][1]))
#         cv2.line(image, p1, p2, (255, 0, 0), 2)
#         cv2.line(image, p1, p3, (0, 255, 0), 2)
#         cv2.circle(image, nose_2d, 1, (0,0,255), 3) #Green
        
        image = draw_axis(image,-x,y,-z,int(nose_2d[0]),int(nose_2d[1]))
        # Add the text on the image
        cv2.putText(image, text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
        cv2.putText(image, "x: " + str(np.round(x,2)), (500, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(image, "y: " + str(np.round(y,2)), (500, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(image, "z: " + str(np.round(z,2)), (500, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


        end = time.time()
        totalTime = end - start

        fps = 1 / totalTime
        print("FPS: ", fps)

        cv2.putText(image, f'FPS: {int(fps)}', (20,450), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

        mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=drawing_spec,
                    connection_drawing_spec=drawing_spec)


    cv2.imshow('Head Pose Estimation', image)

    if cv2.waitKey(5) & 0xFF == 27:
        break


cap.release()

**********************************
 frame 1
 [256. 481.]
**************************************
[[-0.57135268  0.180111   -0.11197725]]
FPS:  0.9716204987336723
**********************************
 frame 2
 [283. 484.]
**************************************


C:\Users\NaDeR\AppData\Local\Temp/ipykernel_3736/3673854249.py:116: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  tvec = np.array([0.0, 0.0, 1.0], np.float) # translation vector


[[-0.59009982  0.16561355 -0.21722674]]
FPS:  1.2870452056438089
**********************************
 frame 3
 [278. 476.]
**************************************
[[-0.55953132  0.1624009  -0.38384101]]
FPS:  2.1371955646979632
**********************************
 frame 4
 [273. 473.]
**************************************
[[-0.52263893  0.1624009  -0.33700994]]
FPS:  2.060963541180413
**********************************
 frame 5
 [228. 477.]
**************************************
[[-0.33036992 -0.05274156  0.03200832]]
FPS:  23.83547102046383
**********************************
 frame 6
 [234. 478.]
**************************************
[[-0.32125359 -0.03626402  0.12567601]]
FPS:  24.576240002343773
**********************************
 frame 7
 [235. 477.]
**************************************
[[-0.33042736 -0.0419228   0.0526562 ]]
FPS:  22.29459259771117
**********************************
 frame 8
 [237. 477.]
**************************************
[[-0.3357396  -0.040479    0.01481055

FPS:  22.999879360831752
**********************************
 frame 55
 [278. 335.]
**************************************
[[-0.37379343 -0.04046722 -0.16729346]]
FPS:  23.407804268238234
**********************************
 frame 56
 [297. 333.]
**************************************
[[-0.43413007 -0.04587272 -0.34060834]]
FPS:  22.80219415797285
**********************************
 frame 57
 [301. 339.]
**************************************
[[-0.38606422 -0.05104255 -0.29949929]]
FPS:  24.47356474755078
**********************************
 frame 58
 [296. 346.]
**************************************
[[-0.42057097 -0.05464871 -0.25022926]]
FPS:  23.836283764200427
**********************************
 frame 59
 [291. 359.]
**************************************
[[-0.41316994 -0.04825998 -0.38383055]]
FPS:  23.2816408093031
**********************************
 frame 60
 [275. 391.]
**************************************
[[-0.39107705 -0.02482311 -0.09845694]]
FPS:  22.78311976838297
********

**********************************
 frame 108
 [301. 398.]
**************************************
[[-0.40975189  0.19120694 -0.29621324]]
FPS:  24.435211185551996
**********************************
 frame 109
 [292. 391.]
**************************************
[[-0.42153365  0.19170783 -0.25886217]]
FPS:  23.890728061881273
**********************************
 frame 110
 [291. 390.]
**************************************
[[-0.42556717  0.19170783 -0.29865407]]
FPS:  23.96758838621935
**********************************
 frame 111
 [291. 390.]
**************************************
[[-0.42790516  0.06394471 -0.20295569]]
FPS:  23.308422433147353
**********************************
 frame 112
 [288. 384.]
**************************************
[[-0.46267808 -0.04849876 -0.45441659]]
FPS:  22.211358003770467
**********************************
 frame 113
 [289. 382.]
**************************************
[[-0.45658375 -0.04267151 -0.43369383]]
FPS:  24.057909167039497
***********************

...

FPS:  24.2786341508593
**********************************
 frame 15510
 [254. 378.]
**************************************
[[-0.36546478 -0.04976045 -0.05301685]]
FPS:  20.133271251446043
**********************************
 frame 15511
 [253. 379.]
**************************************
[[-0.36709558 -0.0626201  -0.04185018]]
FPS:  24.479421034201003
**********************************
 frame 15512
 [253. 380.]
**************************************
[[-0.39457537 -0.04878425 -0.02726006]]
FPS:  20.53384117534747
**********************************
 frame 15513
 [254. 379.]
**************************************
[[-0.40470236 -0.05538735 -0.04071488]]
FPS:  19.673280237150443
**********************************
 frame 15514
 [253. 379.]
**************************************
[[-0.40715513 -0.04610969 -0.02973058]]
FPS:  20.47640063270128
**********************************
 frame 15515
 [254. 382.]
**************************************
[[-0.36028807 -0.05269274 -0.02010929]]
FPS:  17.70592

FPS:  18.29704145109364
**********************************
 frame 15562
 [255. 365.]
**************************************
[[-0.43274772 -0.03657185 -0.0445234 ]]
FPS:  18.932234374365247
**********************************
 frame 15563
 [257. 366.]
**************************************
[[-0.44682396 -0.02660346 -0.06653145]]
FPS:  25.08374997009784
**********************************
 frame 15564
 [257. 367.]
**************************************
[[-0.44688233 -0.03901386 -0.06004287]]
FPS:  21.10522157870107
**********************************
 frame 15565
 [257. 367.]
**************************************
[[-0.43587538 -0.03608146 -0.04568805]]
FPS:  18.410362430483314
**********************************
 frame 15566
 [258. 364.]
**************************************
[[-0.41604967 -0.04087416 -0.01198783]]
FPS:  19.133463800048354
**********************************
 frame 15567
 [259. 363.]
**************************************
[[-0.41736335 -0.04326011 -0.0564376 ]]
FPS:  21.2433

FPS:  22.713780536015033
**********************************
 frame 15613
 [261. 368.]
**************************************
[[-0.36737511 -0.04305423 -0.06056917]]
FPS:  23.101348857959582
**********************************
 frame 15614
 [262. 369.]
**************************************
[[-0.35434196 -0.04737451 -0.02055896]]
FPS:  23.289267945984363
**********************************
 frame 15615
 [260. 370.]
**************************************
[[-0.35603562 -0.04976045 -0.03437035]]
FPS:  19.338303571840232
**********************************
 frame 15616
 [258. 372.]
**************************************
[[-0.35964313 -0.04976045 -0.04499831]]
FPS:  20.630396395598773
**********************************
 frame 15617
 [258. 371.]
**************************************
[[-0.37364631 -0.04737451 -0.06407757]]
FPS:  25.08615038637288
**********************************
 frame 15618
 [253. 371.]
**************************************
[[-0.39692948 -0.04417198 -0.01854166]]
FPS:  19.41

FPS:  16.04983737037462
**********************************
 frame 15663
 [254. 369.]
**************************************
[[-0.38690879 -0.04737451 -0.05104139]]
FPS:  19.15837170211211
**********************************
 frame 15664
 [254. 368.]
**************************************
[[-0.39757352 -0.04417198 -0.03920546]]
FPS:  23.20551492149204
**********************************
 frame 15665
 [254. 369.]
**************************************
[[-0.38603536 -0.05382672 -0.02690349]]
FPS:  19.578052139006232
**********************************
 frame 15666
 [253. 369.]
**************************************
[[-0.38698683 -0.05501876 -0.05482412]]
FPS:  24.471851242466144
**********************************
 frame 15667
 [254. 368.]
**************************************
[[-0.39201133 -0.04823825 -0.02946479]]
FPS:  17.596066553115797
**********************************
 frame 15668
 [253. 367.]
**************************************
[[-0.38973649 -0.04417198 -0.01673439]]
FPS:  17.9964

FPS:  17.61846233980081
**********************************
 frame 15714
 [254. 365.]
**************************************
[[-0.40173429 -0.04178603 -0.04164211]]
FPS:  20.742214815217768
**********************************
 frame 15715
 [255. 365.]
**************************************
[[-0.40400913 -0.04178603 -0.0117862 ]]
FPS:  18.22018149356432
**********************************
 frame 15716
 [254. 365.]
**************************************
[[-0.40228341 -0.04823825 -0.04223913]]
FPS:  22.227955165743662
**********************************
 frame 15717
 [254. 365.]
**************************************
[[-0.39718601 -0.04737451 -0.03855507]]
FPS:  24.47356474755078
**********************************
 frame 15718
 [254. 365.]
**************************************
[[-0.40916817 -0.04417198 -0.01673439]]
FPS:  24.47256517375779
**********************************
 frame 15719
 [254. 364.]
**************************************
[[-0.41100849 -0.04417198 -0.01569694]]
FPS:  23.89168